### TMA4320 - Project 2: Physics


# 0.0 Motivation

In [8]:
# Importing libraries
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# Parameters
sigma_0 = 1000 # kg/m^2
sigma = 500 # kg/m^2
R = 10 # m, Radius
A_s = 0.5*np.pi*(R**2) # Cross-section of boat
g = 9.81 # m/s^2

In [10]:
def newton(f, df, x0, tol=1.e-8, max_iter=30):
    ''' Solve f(x)=0 by Newtons method, shamelessly copied from the note
    "Numerical Solution of Nonlinear Equations" by Anne Kværnø
        The output of each iteration is printed
        Input:
        f, df:   The function f and its derivate f'.
        x0:  Initial values
        tol: The tolerance
      Output:
        The root and the number of iterations
    '''
    x = x0
    print(f"k ={0:3d}, x = {x:18.15f}, f(x) = {f(x):10.3e}")
    for k in range(max_iter):
        fx = f(x)
        if abs(fx) < tol:           # Accept the solution 
            break 
        x = x - fx/df(x)            # Newton-iteration
        print(f"k ={k+1:3d}, x = {x:18.15f}, f(x) = {f(x):10.3e}")
    return x, k+1

In [11]:
def Excercise_1a():
    f = lambda beta: beta - np.sin(beta) -np.pi * sigma/sigma_0
    df = lambda beta: 1 - np.cos(beta)
    beta, iters = newton(f,df,5)
    print("Beta = ",np.degrees(beta),"degrees")
Excercise_1a()
beta = 132.3464590340394
#Define the global variables for the equilibrium, as requested in 1b)
y_M_0 = R*np.cos(beta/2)
y_C_0 = y_M_0 - 4*R/(3*np.pi)
y_B_0 = y_M_0 - 4*R*(np.sin(beta/2))**3 /(3*(beta-np.sin(beta)))
y_D_0 = y_M_0 - R

k =  0, x =  5.000000000000000, f(x) =  4.388e+00
k =  1, x = -1.125780126106931, f(x) = -1.794e+00
k =  2, x =  2.024151652930861, f(x) = -4.456e-01
k =  3, x =  2.334048802844799, f(x) =  4.066e-02
k =  4, x =  2.310007184357851, f(x) =  2.104e-04
k =  5, x =  2.309881463499782, f(x) =  5.840e-09
Beta =  132.3464590340394 degrees


# 1.3 Euler metoden
...

In [ ]:
def euler_step(f, t, w, h_step):
    '''
    Tar inn en vektor-funksjon på formen f(t, _w_) = _y'_ = _y_ + _g(x)_, med nåværende tid (t) og 
    tilstand (w), og regner ut ett Euler-steg.
    Inn:
        f: s-array, som beskriver et system av førsteordens diff-ligninger.
        t: float, nåvernde tid
        w: s-array, som beskriver systemet
        h_step: steglengde
    Ut:
        t_new: neste tidssteg (t+h)
        w_new: systemet etter ett beregnet tidssteg    
    '''
    w_new = w + h_step*f(t, w)
    return w_new


def euler_method(f, t_0, t_end, w_0, h_step):
    
    #Initializing:
    w_num = np.array([w_0])    # Array for the solution w
    t_num = np.arange(t_0, t_end + h_step, h_step)    # Array for the t-values

    for i in range(len(t_num)):
        w_n = euler_step(f, t_num[i], w_num[i], h_step)
        w_num = np.concatenate((w_num, np.array([w_n])))

    return t_num, w_num

## 1.3 Runge Kutta metoden

Implementerer fjerde ordens Runge Kutta.


In [19]:
def rk4_step(f, t, w, h):
    '''
    Tar inn en vektor-funksjon på formen f(t, _w_) = _y'_ = _y_ + _g(x)_, med nåværende tid (t) og 
    tilstand (w), og regner ut ett Runge Kutta steg.
    Inn:
        f: s-array, som beskriver et system av førsteordens diff-ligninger.
        t: float, nåvernde tid
        w: s-array, som beskriver systemet
    Ut:
        t_new: neste tidssteg (t+h)
        w_new: systemet etter ett beregnet tidssteg    
    '''
    k_1 = f(t, w)
    k_2 = f(t + 0.5*h, w + h*k_1*0.5)
    k_3 = f(t + 0.5*h, w + h*k_2*0.5)
    k_4 = f(t + 1.0*h, w + h*k_3)

    w_new = w + h*(1/6) * (k_1 + 2*k_2 + 2*k_3 + k_4)
    t_new = t + h

    return t_new, w_new




# X.0 Conclusion

Lorem Ipsum...